<a href="https://colab.research.google.com/github/anonyms/Kaggle_Titanic/blob/master/Kaggle_Titanic_Machine_Learning_for_Disaster_Colabratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
##!pip install kaggle

In [2]:
import numpy as np 
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import skew
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax

from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from mlxtend.classifier import StackingCVClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from xgboost import XGBRegressor
from xgboost import XGBClassifier

from google.colab import files
from google.colab import data_table
seed = 4

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)


Preparing folders

In [0]:
titanic_data_loctaion = "~/data/titanic/"
submission_file_name = "submissions.csv"
submission_file_location = titanic_data_loctaion + submission_file_name
!mkdir ~/data

Kaggle Set-up

In [4]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anoynms","key":"4b2c6afaefabfef48178035ff45d414b"}'}

In [5]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c titanic -p $titanic_data_loctaion

  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 6.57MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 28.8MB/s]
  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 52.8MB/s]


In [6]:
## Check files are there
!ls -l $titanic_data_loctaion

total 92
-rw-r--r-- 1 root root  3258 Feb 25 20:55 gender_submission.csv
-rw-r--r-- 1 root root 28629 Feb 25 20:56 test.csv
-rw-r--r-- 1 root root 61194 Feb 25 20:56 train.csv


Read data

In [0]:
train = pd.read_csv(titanic_data_loctaion + "train.csv")
test = pd.read_csv(titanic_data_loctaion + "test.csv")
output_sample = pd.read_csv(titanic_data_loctaion + "gender_submission.csv")
train.drop(["PassengerId"],axis=1,inplace=True)
test.drop(["PassengerId"],axis=1,inplace=True)

In [8]:
train.shape, test.shape

((891, 11), (418, 10))

In [0]:
train_copy = train.copy()
y = train_copy['Survived'].reset_index(drop=True)
train_features = train_copy.drop(['Survived'],axis=1)
features = pd.concat([train_features, test]).reset_index(drop=True)

In [0]:
## Output - Data Engineering
##New Features
features['FamilySize'] = features.SibSp + features.Parch + 1
features['isAlone'] = features.FamilySize==1
features['hasCabin'] = ~features.Cabin.isna()
features['isKid'] = features.Age<=10.0
titles = [i.split(",")[1].split(".")[0].strip() for i in features["Name"]]
features["Title"] = pd.Series(titles)


## Fill NAs
features['Embarked'].fillna(features.Embarked.mode()[0],inplace=True)
features['Age'].fillna(features.groupby(['Pclass','Sex']).Age.transform('median'),inplace=True)
features["Fare"] = features["Fare"].fillna(0)
features['Cabin'] = features['Cabin'].fillna("None") 

##Change type
features['Pclass'] = features['Pclass'].astype(int)
features['isAlone'] = features.isAlone.astype(int)
features['hasCabin'] = features.hasCabin.astype(int)
features['isKid'] = features.isKid.astype(int)
features['GenderCode'] = features.Sex.map({'male':0,'female':1})
features["Title"] = features["Title"].replace(['Lady', 'the Countess','Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
features["Title"] = features["Title"].map({"Master":0, "Miss":1, "Ms" : 1 , "Mme":1, "Mlle":1, "Mrs":1, "Mr":2, "Rare":3})
features["Title"] = features["Title"].astype(int)

## New Feature after NA
features['Cabintype'] = features['Cabin'].str[0]

## Normalize & Bin
features['Fare'] = boxcox1p(features['Fare'],boxcox_normmax(features['Fare']+1))
features['CabintypeMapped'] = features['Cabintype'].map( {'A': 1, 'B': 2,'C': 3,'D': 4,'E': 5,'F': 6,'G': 7,'T': 8,'N': 0} ).astype(int)
features['AgeBins'] = pd.cut(features['Age'], 8,labels=False)
features['EmbarkedMapped'] = features['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
features['FareBins'] = pd.cut(features['Fare'],10,labels=False)
##features['Age'] = boxcox1p(features['Age'],boxcox_normmax(features['Age']+1)) Works wore


In [11]:
features.Embarked.mode()[0]

'S'

In [12]:
data_table.DataTable(features,include_index=False,max_columns=20)

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,FamilySize,isAlone,hasCabin,isKid,Title,GenderCode,Cabintype,CabintypeMapped,AgeBins,EmbarkedMapped,FareBins
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,1.984641,None,S,2,0,0,0,2,0,N,0,2,0,3
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,3.784735,C85,C,2,0,1,0,1,1,C,3,3,1,7
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,2.053954,None,S,1,1,0,0,1,1,N,0,2,0,3
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,3.557480,C123,S,2,0,1,0,1,1,C,3,3,0,6
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,2.066179,None,S,1,1,0,0,2,0,N,0,3,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3,"Spector, Mr. Woolf",male,25.0,0,0,A.5. 3236,2.066179,None,S,1,1,0,0,2,0,N,0,2,0,3
1305,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,4.106569,C105,C,1,1,1,0,3,1,C,3,3,1,7
1306,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,1.984641,None,S,1,1,0,0,2,0,N,0,3,0,3
1307,3,"Ware, Mr. Frederick",male,25.0,0,0,359309,2.066179,None,S,1,1,0,0,2,0,N,0,2,0,3


In [13]:
features.describe()

,Pclass,Age,SibSp,Parch,Fare,FamilySize,isAlone,hasCabin,isKid,Title,GenderCode,CabintypeMapped,AgeBins,EmbarkedMapped,FareBins
count,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000
mean,2.294882,29.261398,0.498854,0.385027,2.708232,1.883881,0.603514,0.225363,0.065699,1.576012,0.355997,0.757830,2.405653,0.394194,4.545455
std,0.837836,13.218275,1.041658,0.865560,0.806312,1.583639,0.489354,0.417981,0.247850,0.618081,0.478997,1.564669,1.323626,0.653499,1.629371
min,1.000000,0.170000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,22.000000,0.000000,0.000000,2.051072,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,0.000000,3.000000
50%,3.000000,26.000000,0.000000,0.000000,2.529021,1.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,0.000000,4.000000
75%,3.000000,36.000000,1.000000,0.000000,3.142640,2.000000,1.000000,0.000000,0.000000,2.000000,1.000000,0.000000,3.000000,1.000000,6.000000
max,3.000000,80.000000,8.000000,9.000000,5.224638,11.000000,1.000000,1.000000,1.000000,3.000000,1.000000,8.000000,7.000000,2.000000,9.000000


In [14]:
features.isna().sum()

Pclass             0
Name               0
Sex                0
Age                0
SibSp              0
Parch              0
Ticket             0
Fare               0
Cabin              0
Embarked           0
FamilySize         0
isAlone            0
hasCabin           0
isKid              0
Title              0
GenderCode         0
Cabintype          0
CabintypeMapped    0
AgeBins            0
EmbarkedMapped     0
FareBins           0
dtype: int64

In [15]:
features.dtypes

Pclass               int64
Name                object
Sex                 object
Age                float64
SibSp                int64
Parch                int64
Ticket              object
Fare               float64
Cabin               object
Embarked            object
FamilySize           int64
isAlone              int64
hasCabin             int64
isKid                int64
Title                int64
GenderCode           int64
Cabintype           object
CabintypeMapped      int64
AgeBins              int64
EmbarkedMapped       int64
FareBins             int64
dtype: object

In [0]:
##features.drop(["Cabin","Ticket","Sex","SibSp","Parch"],axis=1,inplace=True)
picked_features = ['Pclass','GenderCode','isAlone','CabintypeMapped','hasCabin','AgeBins','isKid','FamilySize','FareBins','EmbarkedMapped','Title']
##features.drop(["Cabin","Ticket","SibSp","Parch","GenderCode","isKid","isAlone"],axis=1,inplace=True)
features_picked = features[picked_features]

In [17]:
features_final = pd.get_dummies(features_picked)
features_final


,Pclass,GenderCode,isAlone,CabintypeMapped,hasCabin,AgeBins,isKid,FamilySize,FareBins,EmbarkedMapped,Title
0,3,0,0,0,0,2,0,2,3,0,2
1,1,1,0,3,1,3,0,2,7,1,1
2,3,1,1,0,0,2,0,1,3,0,1
3,1,1,0,3,1,3,0,2,6,0,1
4,3,0,1,0,0,3,0,1,3,0,2
...,...,...,...,...,...,...,...,...,...,...,...
1304,3,0,1,0,0,2,0,1,3,0,2
1305,1,1,1,3,1,3,0,1,7,1,3
1306,3,0,1,0,0,3,0,1,3,0,2
1307,3,0,1,0,0,2,0,1,3,0,2


In [0]:
X = features_final.iloc[:len(y), :]
X_sub = features_final.iloc[len(y):, :]

In [19]:
## Voting Classifier
ada = AdaBoostClassifier()
gnb = GaussianNB()
xgbc = XGBClassifier()
mlp = MLPClassifier(hidden_layer_sizes=(30,20,20),  activation='relu', solver='adam', alpha=0.001, batch_size='auto',
    learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
    random_state=21, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
parameters = [{'n_estimators':[5,10,20,100],'max_depth':[3,4,5,6],'criterion':['gini','entropy']}]
gscv = GridSearchCV(estimator=rf, param_grid=parameters, scoring='accuracy',cv=2)
svc = make_pipeline(StandardScaler(),SVC(random_state=1))
r=[0.0001,0.001,0.1,1,10,50,100]
parameters=[{'svc__C':r, 'svc__kernel':['linear']},
      {'svc__C':r, 'svc__gamma':r, 'svc__kernel':['rbf']}]
gscv2 = GridSearchCV(estimator=svc, param_grid=parameters, scoring='accuracy',cv=2)
lr = LogisticRegression(max_iter=10000)
xgb = XGBRegressor(learning_rate=0.01,n_estimators=3460,
                                     max_depth=3, min_child_weight=0,
                                     gamma=0, subsample=0.7,
                                     colsample_bytree=0.7,
                                     objective='reg:squarederror', nthread=-1,
                                     scale_pos_weight=1, seed=27,
                                     reg_alpha=0.00006)

voting = VotingClassifier(estimators=[('gscv', gscv), ('gscv2', gscv2),
('svc', svc), ('ada',ada),('mlp',mlp)], voting='hard', n_jobs=4)

for clf, label in zip([gscv, gscv2, svc, ada, mlp,voting], 
                      ['Grid: Random Forrest',
                       'Grid: SVC',
                       'SVC',
                       'Ada',
                       'MLP',
                       'Voting']):

    scores = model_selection.cross_val_score(clf, X.to_numpy(), y.to_numpy(), cv=3, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]"  % (scores.mean(), scores.std(), label))

voting.fit(X.to_numpy(),y.to_numpy())
predictions = voting.predict(X_sub.to_numpy())

Accuracy: 0.83 (+/- 0.02) [Grid: Random Forrest]
Accuracy: 0.82 (+/- 0.00) [Grid: SVC]
Accuracy: 0.83 (+/- 0.01) [SVC]
Accuracy: 0.82 (+/- 0.02) [Ada]
Accuracy: 0.81 (+/- 0.01) [MLP]
Accuracy: 0.82 (+/- 0.01) [Voting]


In [20]:
submission = pd.read_csv(titanic_data_loctaion + "gender_submission.csv")
submission.iloc[:,1] = predictions
submission.head(15)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [21]:
submission.to_csv(submission_file_location, index=False)
!head $submission_file_location

PassengerId,Survived
892,0
893,1
894,0
895,0
896,1
897,0
898,1
899,0
900,1


In [22]:
comment = input("Enter what comment to put with the Kaggle commit")
comment ="\""+comment+"\""
!kaggle competitions submit -c titanic -f $submission_file_location -m $comment

Enter what comment to put with the Kaggle commitGithub test
100% 2.77k/2.77k [00:00<00:00, 13.3kB/s]
Successfully submitted to Titanic: Machine Learning from Disaster